In [21]:
from utils.long_residuals_mlp import *
from utils.utils import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Reshape
import cv2
from sklearn.svm import SVC
import optuna
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")


In [ ]:

PATCH_SIZE = 32
CLASSES      = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding']

MODEL_WEIGHTS = '/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/long_residuals-4096-32-1e-05/long_residuals-4096-32-1e-05_best.h5'
images_directory = '/home/georg/projects/university/C3_ML_for_CV/MIT_split/'


model = build_long_residuals_mlp(input_shape=(PATCH_SIZE, PATCH_SIZE, 3),
                                  activation='relu', 
                                  regularization_coeff=0.00001, 
                                  num_units=4096, 
                                  filename='myMLP.png')

model.load_weights(MODEL_WEIGHTS)

model_layer = Model(inputs=model.input, outputs=model.get_layer('dense_38').output)



for subdir1 in os.listdir(images_directory):
    for subdir2 in os.listdir(os.path.join(images_directory, subdir1)):
        for file in os.listdir(os.path.join(images_directory, subdir1, subdir2)):
            img = load_and_preprocess_image(image_path=os.path.join(images_directory, subdir1, subdir2, file),
                                            target_size=(32, 32,3))
            
            layer_output = model_layer.predict(img)
            layer_output = np.array(layer_output)
            print(layer_output.shape)
            # save np array to csv file
            np.savetxt(os.path.join('/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/embeddings/image', file.split('.')[0]+'.csv'), layer_output, delimiter=',')

In [2]:
EMBEDDINGS_DIR = '/image'
PATCH_SIZE = 32

SVM_C = 1e-3
SVM_KERNEL = 'linear'
SVM_GAMMA = 'scale'


In [7]:
train_images_filenames = pickle.load(open('../MIT_split/train_images_filenames.dat','rb'))
test_images_filenames = pickle.load(open('../MIT_split/test_images_filenames.dat','rb'))
train_images_filenames = ['..' + n[15:] for n in train_images_filenames]
test_images_filenames  = ['..' + n[15:] for n in test_images_filenames]
train_labels = pickle.load(open('../MIT_split/train_labels.dat','rb')) 
test_labels = pickle.load(open('../MIT_split/test_labels.dat','rb'))

embeddings_dict = read_embeddings(directory='./image')

training_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in train_images_filenames])
test_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in test_images_filenames])

print(training_embeddings.shape)
print(test_embeddings.shape)

(1881, 256)
(807, 256)


In [9]:
def split_into_K_folds(x, y, k=5):
    k = 5  # number of folds
    kf = KFold(n_splits=k, shuffle=True)

    folds = []
    for train_index, test_index in kf.split(x):
        train_data = [x[i] for i in train_index]
        train_labels = [y[i] for i in train_index]
        test_data = [x[i] for i in test_index]
        test_labels = [y[i] for i in test_index]
        folds.append((train_data, train_labels, test_data, test_labels))
    return folds

In [10]:
print('Preparing k-fold cross validation...')

all_data = train_images_filenames + test_images_filenames
all_labels = train_labels + test_labels

k = 5  # number of folds
# split data into 5 folds using custom function
folds = split_into_K_folds(all_data, all_labels, k)

# Now each fold in the 'folds' list contains the training and test data for that fold in the format: (train_data, train_labels, test_data, test_labels)
for fold_index, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    print(f"Fold {fold_index + 1}:")
    print(f"Number of samples in train set: {len(train_data)}")
    print(f"Number of samples in test set: {len(test_data)}")
    print(f"Number of labels in train set: {len(train_labels)}")
    print(f"Number of labels in test set: {len(test_labels)}")
    print()

Preparing k-fold cross validation...
Fold 1:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 2:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 3:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 4:
Number of samples in train set: 2151
Number of samples in test set: 537
Number of labels in train set: 2151
Number of labels in test set: 537

Fold 5:
Number of samples in train set: 2151
Number of samples in test set: 537
Number of labels in train set: 2151
Number of labels in test set: 537



In [24]:
print('Running GridSearch on SVM parameters...')

gamma_vals = [0.05, 0.1, 0.15, 0.2, 0.25]
c_vals = [0.01, 0.05, 0.1, 0.5, 1, 10, 100]
kernel_vals = ['linear', 'rbf', 'poly', 'sigmoid']

def objective(trial):

    svm_kernel = trial.suggest_categorical('svm_kernel', kernel_vals)
    svm_C = trial.suggest_categorical('svm_C', c_vals)
    svm_gamma = trial.suggest_categorical('svm_gamma', gamma_vals)


    metrics = []
    # Run k-fold cross validation per hyperparameter combination
    for i, fold in enumerate(folds, 0):

        train_data, train_labels, test_data, test_labels = fold
        
        train_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in train_data])
        test_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in test_data])
        

        # fit SVM classifier
        print(f'Fold {i}: fitting svm with C={svm_C}, kernel={svm_kernel}, gamma={svm_gamma}')
        svm = SVC(C=svm_C, kernel=svm_kernel, gamma=svm_gamma)
        svm.fit(train_embeddings, train_labels) 
        # predict labels for test data
        pred_labels = svm.predict(test_embeddings)
    
        # get accuracy metric for current run
        metrics.append(get_metrics(test_labels, pred_labels)[0])

    return sum(metrics)/len(metrics)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

study.best_params

[I 2024-01-14 22:47:13,975] A new study created in memory with name: no-name-6f5c7651-efdc-4165-bc2b-80df53fbea10


Running GridSearch on SVM parameters...
Fold 0: fitting svm with C=0.5, kernel=rbf, gamma=0.05
Fold 1: fitting svm with C=0.5, kernel=rbf, gamma=0.05
Fold 2: fitting svm with C=0.5, kernel=rbf, gamma=0.05
Fold 3: fitting svm with C=0.5, kernel=rbf, gamma=0.05
Fold 4: fitting svm with C=0.5, kernel=rbf, gamma=0.05


[I 2024-01-14 22:47:16,845] Trial 0 finished with value: 0.6365399126359439 and parameters: {'svm_kernel': 'rbf', 'svm_C': 0.5, 'svm_gamma': 0.05}. Best is trial 0 with value: 0.6365399126359439.


Fold 0: fitting svm with C=10, kernel=linear, gamma=0.05
Fold 1: fitting svm with C=10, kernel=linear, gamma=0.05
Fold 2: fitting svm with C=10, kernel=linear, gamma=0.05
Fold 3: fitting svm with C=10, kernel=linear, gamma=0.05
Fold 4: fitting svm with C=10, kernel=linear, gamma=0.05


[I 2024-01-14 22:47:21,409] Trial 1 finished with value: 0.5539317286591487 and parameters: {'svm_kernel': 'linear', 'svm_C': 10, 'svm_gamma': 0.05}. Best is trial 0 with value: 0.6365399126359439.


...

Fold 0: fitting svm with C=100, kernel=linear, gamma=0.05
Fold 1: fitting svm with C=100, kernel=linear, gamma=0.05
Fold 2: fitting svm with C=100, kernel=linear, gamma=0.05
Fold 3: fitting svm with C=100, kernel=linear, gamma=0.05
Fold 4: fitting svm with C=100, kernel=linear, gamma=0.05


[I 2024-01-14 22:55:02,130] Trial 148 finished with value: 0.5304874249755975 and parameters: {'svm_kernel': 'linear', 'svm_C': 100, 'svm_gamma': 0.05}. Best is trial 45 with value: 0.648811031961953.


Fold 0: fitting svm with C=1, kernel=rbf, gamma=0.05
Fold 1: fitting svm with C=1, kernel=rbf, gamma=0.05
Fold 2: fitting svm with C=1, kernel=rbf, gamma=0.05
Fold 3: fitting svm with C=1, kernel=rbf, gamma=0.05
Fold 4: fitting svm with C=1, kernel=rbf, gamma=0.05


[I 2024-01-14 22:55:04,750] Trial 149 finished with value: 0.648811031961953 and parameters: {'svm_kernel': 'rbf', 'svm_C': 1, 'svm_gamma': 0.05}. Best is trial 45 with value: 0.648811031961953.


{'svm_kernel': 'rbf', 'svm_C': 1, 'svm_gamma': 0.05}

In [27]:
study.best_params

{'svm_kernel': 'rbf', 'svm_C': 1, 'svm_gamma': 0.05}